## Embed validation and test sets of individually annotated Ovenbird songs with HawkEars

In [ ]:
import bioacoustics_model_zoo as bmz
import pandas as pd
import numpy as np
from pathlib import Path

load tables listing the clips to embed for validation and test sets

In [ ]:
annotated_dataset_path = "../../../localization_dataset_labeled_clips"
labels = pd.read_csv(f"{annotated_dataset_path}/labeled_clips.csv")
val = labels[labels["data_split"] == "val"]
test = labels[labels["data_split"] == "test"]

# only predict on the 4-7 s part of the 10s clip
val = pd.DataFrame({"file": val["rel_path"], "start_time": 4, "end_time": 7}).set_index(
    ["file", "start_time", "end_time"]
)
test = pd.DataFrame(
    {"file": test["rel_path"], "start_time": 4, "end_time": 7}
).set_index(["file", "start_time", "end_time"])
len(val), len(test)

(2739, 1224)

In [ ]:
# load the HawkEars model from the Bioacoustics Model Zoo
hawkears = bmz.HawkEars()

File hgnet1.ckpt already exists; skipping download.
Loading model from local checkpoint /Users/SML161/song25_oven_aiid/oven_aiid/develop_and_evaluate_aiid/5_evaluate_individual_id/hgnet1.ckpt...
File hgnet2.ckpt already exists; skipping download.
Loading model from local checkpoint /Users/SML161/song25_oven_aiid/oven_aiid/develop_and_evaluate_aiid/5_evaluate_individual_id/hgnet2.ckpt...
File hgnet3.ckpt already exists; skipping download.
Loading model from local checkpoint /Users/SML161/song25_oven_aiid/oven_aiid/develop_and_evaluate_aiid/5_evaluate_individual_id/hgnet3.ckpt...
File hgnet4.ckpt already exists; skipping download.
Loading model from local checkpoint /Users/SML161/song25_oven_aiid/oven_aiid/develop_and_evaluate_aiid/5_evaluate_individual_id/hgnet4.ckpt...
File hgnet5.ckpt already exists; skipping download.
Loading model from local checkpoint /Users/SML161/song25_oven_aiid/oven_aiid/develop_and_evaluate_aiid/5_evaluate_individual_id/hgnet5.ckpt...


/Users/SML161/miniconda3/envs/opso0110/lib/python3.11/site-packages/opensoundscape/preprocess/preprocessors.py:512: DeprecationWarning: sample_shape argument is deprecated. Please use height, width, channels arguments instead. 
                The current behavior is to override height, width, channels with sample_shape 
                when sample_shape is not None.
                
  warnings.warn(
/Users/SML161/miniconda3/envs/opso0110/lib/python3.11/site-packages/opensoundscape/ml/cnn.py:599: UserWarning: 
                    This architecture is not listed in opensoundscape.ml.cnn_architectures.ARCH_DICT.
                    It will not be available for loading after saving the model with .save() (unless using pickle=True). 
                    To make it re-loadable, define a function that generates the architecture from arguments: (n_classes, n_channels) 
                    then use opensoundscape.ml.cnn_architectures.register_architecture() to register the generating function.

generate embeddings on the validation and test dataframes

In [ ]:
test_embeddings = hawkears.embed(
    test, batch_size=128, num_workers=9, audio_root=annotated_dataset_path
)
# test_embeddings.to_csv(f"../resources/hawkears_test_embeddings.csv")

Path("../resources/precomputed_embeddings").mkdir(exist_ok=True)
np.save(
    "../resources/precomputed_embeddings/hawkears_test_embeddings.npy",
    test_embeddings.values,
)
val_embeddings = hawkears.embed(
    val, batch_size=128, num_workers=0, audio_root=annotated_dataset_path
)
# val_embeddings.to_csv(f"../resources/hawkears_val_embeddings.csv")
np.save(
    "../resources/precomputed_embeddings/hawkears_val_embeddings.npy",
    val_embeddings.values,
)

  0%|          | 0/10 [00:00<?, ?it/s]

/Users/SML161/miniconda3/envs/opso0110/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/22 [00:00<?, ?it/s]

/Users/SML161/miniconda3/envs/opso0110/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
